In [2]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Install image package
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

# pip installs
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:5 http://ppa.launchpad.net/jonathonf/ffmpeg-4/ubuntu bionic InRelease [15.9 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main amd64 Packages [38.5 kB]
Get:12 http://ppa.launchpad.net/jonathonf/ffmpeg-4/ubuntu bionic/main amd64 Packages [12.5 kB]
Ign:13 https://developer.download.nvidia.com/compute/ma

In [4]:
# Imports
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from datasets import load_from_disk
import evaluate 
from transformers import WhisperForConditionalGeneration, WhisperProcessor, Seq2SeqTrainingArguments, Seq2SeqTrainer, WhisperTokenizer, WhisperFeatureExtractor
from huggingface_hub import notebook_login

In [5]:
# Connect to huggingface
# If this doesn't work (I couldn't interact with the widget), use 'huggingface-cli login' instead
# Get the token from Account->Settings->Acess tokens
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [6]:
# Load pre-processed data
# original (tiny-15): common_voice_processed.hf
# tiny-25: common_voice_processed_tiny_25.hf
# base-25: common_voice_processed_base_25.hf
common_voice = load_from_disk("/content/drive/MyDrive/Scalable/lab2/common_voice_processed_base_25.hf")

In [7]:
# Load model
# To improve performance, one could go for a larger model, but this would require a lot more time and resources to train
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="Dutch", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="Dutch", task="transcribe")

Downloading:   0%|          | 0.00/185k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/827 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/494k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

In [8]:
# To improve performance, one could go for a larger model, but this would require a lot more time and resources to train
processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="Dutch", task="transcribe")

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

# Handle the padding of the data
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# Load the evaluation metric
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

# Load a pre-trained checkpoint for Whisper
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

# Some model configs
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

# Define training config, including pushing the model to HuggingFace
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-base-dutch-25",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=2000,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=50,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

# Setup trainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

# Save processor config as a file
processor.save_pretrained(training_args.output_dir)

Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/290M [00:00<?, ?B/s]

Cloning https://huggingface.co/renesteeman/whisper-base-dutch-25 into local empty directory.
max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend
Feature extractor saved in ./whisper-base-dutch-25/preprocessor_config.json
tokenizer config file saved in ./whisper-base-dutch-25/tokenizer_config.json
Special tokens file saved in ./whisper-base-dutch-25/special_tokens_map.json
added tokens file saved in ./whisper-base-dutch-25/added_tokens.json


In [9]:
# Model training, this takes *time* (luckily allows for checkpointing)
try:
  trainer.train(resume_from_checkpoint=True)
  print("Using checkpoint")
except:
  trainer.train()
  print("Starting new model")

# Set args for saving the model on Hugging Face
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: nl, split: test",
    "language": "nl",
    "model_name": "Whisper Base Dutch 25",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-base",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

# Push to Hugging Face
trainer.push_to_hub(**kwargs)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10264
  Num Epochs = 4
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 2000
  Number of trainable parameters = 72593920


Step,Training Loss,Validation Loss,Wer
500,0.370400,0.543751,33.988989
1000,0.235600,0.505862,31.351567
1500,0.133500,0.495311,30.574549
2000,0.099800,0.491878,29.948495


***** Running Evaluation *****
  Num examples = 2686
  Batch size = 8
Saving model checkpoint to ./whisper-base-dutch-25/checkpoint-500
Configuration saved in ./whisper-base-dutch-25/checkpoint-500/config.json
Model weights saved in ./whisper-base-dutch-25/checkpoint-500/pytorch_model.bin
Feature extractor saved in ./whisper-base-dutch-25/checkpoint-500/preprocessor_config.json
Feature extractor saved in ./whisper-base-dutch-25/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 2686
  Batch size = 8
Saving model checkpoint to ./whisper-base-dutch-25/checkpoint-1000
Configuration saved in ./whisper-base-dutch-25/checkpoint-1000/config.json
Model weights saved in ./whisper-base-dutch-25/checkpoint-1000/pytorch_model.bin
Feature extractor saved in ./whisper-base-dutch-25/checkpoint-1000/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 2686
  Batch size = 8
Saving model checkpoint to ./whisper-base-dutch-25/checkpoint-1500
Configuration saved i

Starting new model


Model weights saved in ./whisper-base-dutch-25/pytorch_model.bin
Feature extractor saved in ./whisper-base-dutch-25/preprocessor_config.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/277M [00:00<?, ?B/s]

Upload file runs/Nov29_16-51-47_121fd245795a/events.out.tfevents.1669740719.121fd245795a.79.0:  28%|##8       …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/renesteeman/whisper-base-dutch-25
   6e6b611..f007478  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/renesteeman/whisper-base-dutch-25
   6e6b611..f007478  main -> main

To https://huggingface.co/renesteeman/whisper-base-dutch-25
   f007478..99a87c6  main -> main

   f007478..99a87c6  main -> main



'https://huggingface.co/renesteeman/whisper-base-dutch-25/commit/f0074783729c439c4240cb8bbb2368dbe6038497'